In [1]:
import sys
sys.path.append("../Python")
import ROOT as R
from array import array
import time
import numpy as np
# Turn jsroot off if you want to make a pdf from this file.
%jsroot on
from root_helpers import SetStyle
from root_helpers import fancy_plot
from root_helpers import print_mc_particle_tree
from root_helpers import print_daughters
R.EnableImplicitMT()
import os
recompile = True
try:
    if os.path.getmtime('../Python/Utility_Functions_C.so') - os.path.getmtime('../Python/Utility_Functions.C') > 0:
        recompile = False
        print("Recompile is not needed")
    else:
        print("Recompiling: ")
except:
    print("Recompile needed, file not found.")
if recompile:
    R.gROOT.LoadMacro("../Python/Utility_Functions.C++")
else:
    R.gSystem.Load("../Python/Utility_Functions_C.so")

R.gSystem.Load("lib/libMC2021")
R.Utility_Functions()

Welcome to JupyROOT 6.29/01
Recompile is not needed


'Utility Functions V1.0.5 \n'

In [2]:
ch = R.TChain("MiniDST")
ch.Add("/data/HPS/data/physrun2021/sim_2021/hpsForward_e-_3.0GeV_z0.0_*_SLIC-v06-00-01_QGSP_BERT_HPS-v2019-3pt7GeV_recon.root")
# ch.Add("/data/HPS/data/physrun2021/pass0/minidst/hps_0147*.root")
print(f"Number of events loaded: {ch.GetEntries()/1e6:7.3f}M")
df = R.RDataFrame(ch)

Number of events loaded:   1.847M


In [3]:
R.gInterpreter.ProcessLine('''auto EAC = Ecal_Analysis_Class();''')   # This is key. It puts the EAC in C++ space.
print(f"{R.EAC.Version()}")

V1.0.4


In [4]:
dfx = R.EAC.extend_dataframe(R.RDF.AsRNode(df));

In [5]:
print("Available data names in Tuple:")
ColumnNames=dfx.GetColumnNames()
ll = 0
pr_colnames = [x for x in ColumnNames if str(x).startswith('')]
for nn in pr_colnames:
    if ll < len(nn):
        ll = len(nn)
for n in range(len(pr_colnames)):
    if n%4 == 0:
        print("")
    print(f"{str(pr_colnames[n]):{ll}s}",end="")

Available data names in Tuple:

ecal_cluster_energy           ecal_cluster_hits             ecal_cluster_nhits            ecal_cluster_seed_energy      
ecal_cluster_seed_index       ecal_cluster_seed_ix          ecal_cluster_seed_iy          ecal_cluster_time             
ecal_cluster_uncor_energy     ecal_cluster_uncor_hits       ecal_cluster_uncor_nhits      ecal_cluster_uncor_seed_energy
ecal_cluster_uncor_seed_index ecal_cluster_uncor_seed_ix    ecal_cluster_uncor_seed_iy    ecal_cluster_uncor_time       
ecal_cluster_uncor_x          ecal_cluster_uncor_y          ecal_cluster_uncor_z          ecal_cluster_x                
ecal_cluster_y                ecal_cluster_z                ecal_hit_energy               ecal_hit_index_x              
ecal_hit_index_y              ecal_hit_time                 event_number                  ext_trigger                   
hodo_cluster_energy           hodo_cluster_ix               hodo_cluster_iy               hodo_cluster_layer            


In [6]:
max_i = 50
hh = dfx.Histo1D("mc_part_primary_index")
hhn = dfx.Define("n","mc_score_primary_hits_e.size()").Histo1D(("hhn","Num primary part.", max_i, -0.5,max_i - 0.5),"n")
hhn2 = dfx.Define("n","mc_score_secondary_hits_e.size()").Histo1D(("hhn2","Num secondary part.", max_i, -0.5,max_i - 0.5),"n")
hhn_hits = dfx.Histo1D(("hhn_hits","Number of hits per primary", max_i, -0.5,max_i - 0.5),"n_mc_score_primary_hits")
hhn2_hits = dfx.Histo1D(("hhn2_hits","Number of hits per secondary", max_i, -0.5,max_i - 0.5),"n_mc_score_secondary_hits")
hhe = dfx.Histo1D(("hhe","Energy of Primary Hit on score plane.",1000,-0.1,3.2),"mc_score_primary_hits_e")
hhe2 = dfx.Histo1D(("hhe2","Energy of Secondary Hit on score plane.",1000,-0.1,3.2),"mc_score_secondary_hits_e")
hhn_clus = dfx.Define("n","return mc_score_cluster_indexes.size()").Histo1D(("hhn_clus","N score clusters", max_i,  -0.5, max_i - 0.5),"n")
hhn_clus_hits = dfx.Define("nn","RVec<int> out;for(size_t i=0; i< mc_score_cluster_indexes.size(); ++i){ out.push_back(mc_score_cluster_indexes[i].size());} return out;").Histo1D(("hhn_clus","N score clusters", max_i,  -0.5, max_i - 0.5),"nn")

In [7]:
%%time
cc0 = R.TCanvas("cc0","CC0",1200,600)
cc0.Divide(2,0)
p1 = cc0.cd(1)
leg0_p1 = R.TLegend(0.70, 0.75 ,0.99, 0.9)
p1.SetLogy()
hhn.SetStats(0)
hhn2_hits.SetStats(0)
hhn2_hits.SetTitle("Numbers")
hhn2_hits.Draw()
hhn.SetLineWidth(3)
hhn.SetLineColor(R.kRed)
hhn.Draw("same")
leg0_p1.AddEntry(hhn.GetPtr(),"N Primary particles")

hhn_clus.SetLineWidth(3)
hhn_clus.SetLineColor(R.kCyan+1)
hhn_clus.Draw("same")
leg0_p1.AddEntry(hhn_clus.GetPtr(),"N score clusters.")

hhn_clus_hits.SetLineColor(R.kCyan+3)
hhn_clus_hits.Draw("same")
leg0_p1.AddEntry(hhn_clus_hits.GetPtr(),"N score cluster hits.")

hhn2.SetLineColor(R.kOrange)
hhn2.SetStats(0)
hhn2.Draw("same")
leg0_p1.AddEntry(hhn2.GetPtr(),"N Secondary particles")
hhn_hits.SetLineColor(R.kBlue)
hhn_hits.Draw("same")
leg0_p1.AddEntry(hhn_hits.GetPtr(),"N hits from primary part.")
hhn2_hits.SetLineColor(R.kGreen)
hhn2_hits.Draw("same")
leg0_p1.AddEntry(hhn2_hits.GetPtr(),"N hits from secondary part.")
leg0_p1.Draw()
p2 = cc0.cd(2)
p2.SetLogy()
hhe2.SetTitle("Energies")
hhe2.SetStats(0)
hhe2.SetLineColor(R.kOrange)
hhe2.Draw()
hhe.SetStats(0)
hhe.SetLineColor(R.kBlue)
hhe.Draw("same")
cc0.Draw()

CPU times: user 37.2 s, sys: 1.31 s, total: 38.5 s
Wall time: 5.65 s


In [10]:
df_tmp = dfx.Define("test_n_second","""
  int count=0;
   for(size_t i_part=0; i_part < mc_part_z.size(); ++i_part){
      if(abs(mc_part_z[i_part]) >= 1e-6 ) {
         for (size_t i = 0; i < mc_score_part_idx.size(); ++i) {
            if ((mc_score_z[i] > 1440) && (mc_score_pz[i] > 0.01) &&
                mc_score_part_idx[i] == i_part) {
                count++;
            }
         }
      }
   }
   return count;""").Define("test_nn_second","""
  vector<int> out;
   for(size_t i_part=0; i_part < mc_part_z.size(); ++i_part){
      if(abs(mc_part_z[i_part]) >= 1e-6 ) {
         int count = 0;
         for (size_t i = 0; i < mc_score_part_idx.size(); ++i) {
            if ((mc_score_z[i] > 1440) && (mc_score_pz[i] > 0.01) &&
                mc_score_part_idx[i] == i_part) {
                count++;
            }
         }
         out.push_back(count);
      }
   }
   return out;""").Define("test_n_first","""
  int count=0;
   for(size_t i_part=0; i_part < mc_part_z.size(); ++i_part){
      if(abs(mc_part_z[i_part]) < 1e-6 ) {
         for (size_t i = 0; i < mc_score_part_idx.size(); ++i) {
            if ((mc_score_z[i] > 1440) && (mc_score_pz[i] > 0.01) &&
                mc_score_part_idx[i] == i_part) {
                count++;
            }
         }
      }
   }
   return count;""").Define("test_nn_first","""
   vector<int> out;
   for(size_t i_part=0; i_part < mc_part_z.size(); ++i_part){
      if(abs(mc_part_z[i_part]) < 1e-6 ) {
         int count=0;
         for (size_t i = 0; i < mc_score_part_idx.size(); ++i) {
            if ((mc_score_z[i] > 1440) && (mc_score_pz[i] > 0.01) &&
                mc_score_part_idx[i] == i_part) {
                count++;
            }
         }
         out.push_back(count);
      }
   }
   return out;""")

In [13]:
h_tmp1 = df_tmp.Histo1D(("h_tmp1","Num ",31,-0.5,30.5),"test_n_first")
h_tmp2 = df_tmp.Histo1D(("h_tmp2","Num second",31,-0.5,30.5),"test_n_second")
h_tmp1n = df_tmp.Histo1D(("h_tmp1n","Num ",31,-0.5,30.5),"test_nn_first")
h_tmp2n = df_tmp.Histo1D(("h_tmp2n","Num second",31,-0.5,30.5),"test_nn_second")
h_tmp1nn = df_tmp.Define("nnn","test_nn_first.size()").Histo1D(("h_tmp1nn","Num ",31,-0.5,30.5),"nnn")
h_tmp2nn = df_tmp.Define("nnn","test_nn_second.size()").Histo1D(("h_tmp2nn","Num second",31,-0.5,30.5),"nnn")

In [14]:
ccx = R.TCanvas("ccx","CCX",800,600)
ccx.SetLogy()
h_tmp1.Draw()
h_tmp2.SetLineColor(R.kRed)
h_tmp2.Draw("same")
h_tmp1n.SetLineColor(R.kGreen)
h_tmp1n.Draw("same")
h_tmp1nn.SetLineColor(R.kBlack)
h_tmp1nn.Draw("same")
h_tmp2n.SetLineColor(R.kOrange)
h_tmp2n.Draw("same")
h_tmp2nn.SetLineColor(R.kYellow)
h_tmp2nn.Draw("same")
ccx.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: ccx


In [18]:
R.gSystem.Load("/data/HPS/lib/libMiniDst")
mdst = R.MiniDst()          # Initiate the class
mdst.use_mc_particles=True  # Tell it to look for the MC Particles in the TTree
mdst.use_ecal_cluster_uncor = True;
mdst.use_mc_scoring =True;
mdst.DefineBranchMap()      # Define the map of all the branches to the contents of the TTree
mdst.SetBranchAddressesOnTree(ch) # Connect the TChain (which contains the TTree) to the class.
print(f"MminiDST version = {mdst._version_()}")

MminiDST version = 1.0.8


Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.
Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.
Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.
Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.
Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.


In [19]:
%%time
hist_e = R.TH1D("hist_e","Score plane energy of primary hit", 1000, -0.1,3.2)
hist_e2 = R.TH1D("hist_e2","Score plane energy of secondary hits", 1000, -0.1, 3.2)
hist_n = R.TH1D("hist_n","Number of hits by primary",21,-0.5,20.5)
hist_n2 = R.TH1D("hist_n","Number of hits by secondary",21,-0.5,20.5)
hist_pn = R.TH1D("hist_pn","Number of  primary",21,-0.5,20.5)
hist_pn2 = R.TH1D("hist_pn2","Number of  secondary",21,-0.5,20.5)
for event in range(ch.GetEntries()):
    ch.GetEntry(event)
    # Find the primary indices:
    p_count = 0
    p_count2 = 0
    for i_part in range(mdst.mc_part_z.size()):
        if abs(mdst.mc_part_z[i_part]) < 1e-6:
            p_count += 1
            e_sum = 0.
            count = 0
            count_secondary = 0.
            for i_score in range(mdst.mc_score_pz.size()):
                if mdst.mc_score_z[i_score] > 1400. and mdst.mc_score_pz[i_score] > 0.01 and mdst.mc_score_part_idx[i_score] == i_part:
                    e_sum += R.TMath.Sqrt(mdst.mc_score_px[i_score]*mdst.mc_score_px[i_score]+mdst.mc_score_py[i_score]*mdst.mc_score_py[i_score]+mdst.mc_score_pz[i_score]*mdst.mc_score_pz[i_score])
                    count += 1
            hist_e.Fill(e_sum)
            hist_n.Fill(count)
        else:
            p_count2 += 1
            e_sum_secondary = 0.
            count_secondary = 0
            for i_score in range(mdst.mc_score_pz.size()):
                if mdst.mc_score_z[i_score] > 1400. and mdst.mc_score_pz[i_score] > 0.01 and mdst.mc_score_part_idx[i_score] == i_part:
                    count_secondary += 1
                    e_sum_secondary += R.TMath.Sqrt(mdst.mc_score_px[i_score]*mdst.mc_score_px[i_score]+mdst.mc_score_py[i_score]*mdst.mc_score_py[i_score]+mdst.mc_score_pz[i_score]*mdst.mc_score_pz[i_score])
            hist_e2.Fill(e_sum_secondary)
            hist_n2.Fill(count_secondary)

    hist_pn.Fill(p_count)
    hist_pn2.Fill(p_count2)

TypeError: none of the 2 overloaded methods succeeded. Full details:
  double& std::vector<double>::operator[](unsigned long __n) =>
    AbortSignal: abort from C++; program state was reset
  const double& std::vector<double>::operator[](unsigned long __n) =>
    ReferenceError: attempt to access a null-pointer

[/Users/maurik/root/master/lib/libCore.so][/Users/maurik/root/master/lib/libCore.so] TUnixSystem::DispatchSignals(ESignals) (no debug info)
 TUnixSystem::DispatchSignals(ESignals) (no debug info)
[/usr/lib/system/libsystem_platform.dylib] _sigtramp (no debug info)
[/usr/lib/system/libsystem_platform.dylib] _sigtramp (no debug info)
[/usr/lib/system/libsystem_pthread.dylib] pthread_kill (no debug info)
[/usr/lib/system/libsystem_pthread.dylib] pthread_kill (no debug info)
[/usr/lib/system/libsystem_c.dylib] abort (no debug info)
[/usr/lib/system/libsystem_c.dylib] abort (no debug info)
[/usr/lib/system/libsystem_malloc.dylib] malloc_vreport (no debug info)
[/usr/lib/system/libsystem_malloc.dylib] malloc_zone_error (no debug info)
[/usr/lib/system/libsystem_malloc.dylib] nanov2_guard_corruption_detected (no debug info)
[/usr/lib/system/libsystem_malloc.dylib] _nanov2_free (no debug info)
[/usr/lib/system/libsystem_malloc.dylib] malloc_vreport (no debug info)
[/usr/lib/system/libsystem_ma

Warning in <TROOT::Append>: Replacing existing TH1: hist_e (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist_e2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist_n (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist_n (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist_pn (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist_pn2 (Potential memory leak).
Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.
Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.
Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.
Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.
Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.
 *** Break *** abort
 *** Break *** abort
 *** Break *** abort


In [12]:
cc1 = R.TCanvas("cc1","CC1", 1200, 600)
cc1.Divide(2,1)
p1 = cc1.cd(1)
p1.SetLogy()
hist_n2.Draw()
hist_pn.SetLineColor(R.kRed)
hist_pn.SetLineWidth(2)
hist_pn.Draw("same")
hist_pn2.SetLineColor(R.kOrange)
hist_pn2.SetLineWidth(3)
hist_pn2.Draw("same")
hist_n.SetLineColor(R.kBlue)
hist_n.Draw("same")
hist_n2.SetLineColor(R.kGreen)
hist_n2.Draw("same")
p2 = cc1.cd(2)
p2.SetLogy()
hist_e.SetStats(0)
hist_e2.SetLineColor(R.kOrange)
hist_e2.Draw()
hist_e.SetLineColor(R.kBlue)
hist_e.Draw("same")
cc1.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc1


61